In [252]:
# from tpk4170.visualization import Ur5Visualizer
from dataclasses import dataclass
import modern_robotics as mr
import numpy as np
import sympy as sp

np.set_printoptions(suppress=True)

PI_HALF = np.pi / 2
PI = np.pi


In [253]:
@dataclass
class DHLink:
    a: float
    alpha: float
    d: float
    theta: float

    def matrix(self) -> np.array:
        ct = np.cos(self.theta)
        st = np.sin(self.theta)
        ca = np.cos(self.alpha)
        sa = np.sin(self.alpha)
        return np.array(
            [
                [ct, -st * ca, st * sa, self.a * ct],
                [st, ct * ca, -ct * sa, self.a * st],
                [0.0, sa, ca, self.d],
                [0.0, 0.0, 0.0, 1.0],
            ]
        )


In [254]:
class KR6:
    d1, d2, d3, d4 = 0.400, 0.455, 0.420, 0.080 #distansene mellom jointsene som ikke er 0
    a1, a2 = 0.025, 0.035 #offsetsene 


    def __init__(self):
        self._M = np.array(
            [
                [0,  0,  1, self.a1+self.d2+self.d3+self.d4],
                [0, -1,  0,                               0],
                [1,  0,  0,                 self.a2+self.d1],
                [0,  0,  0,                               1]
            ]
        )

        self._Slist = np.array(
            [
                [0, 0, 1,                  0,               0,                       0],
                [0, 1, 0,           -self.d1,               0,                 self.a1],
                [0, 1, 0,           -self.d1,               0,         self.a1+self.d2],
                [1, 0, 0,                  0, self.a2+self.d1,                       0],
                [0, 1, 0, -(self.d1+self.a2),               0, self.a1+self.d2+self.d3],
                [1, 0, 0,                  0, self.a2+self.d1,                       0]
            ]
        ).T

        self._Blist = mr.Adjoint(np.linalg.inv(self._M)) @ self._Slist

    def fKinS(self, theta): #Returnerer T_SB, altså transformasjonsmatrisa fra space til end-effector
        return mr.FKinSpace(self._M, self._Slist, theta)


In [255]:
kr6 = KR6()

sp.Matrix(kr6._Blist.T)


Matrix([
[1.0,  0.0, 0.0,    0.0, -0.98,   0.0],
[0.0, -1.0, 0.0, -0.955,   0.0, 0.035],
[0.0, -1.0, 0.0,   -0.5,   0.0, 0.035],
[0.0,  0.0, 1.0,    0.0,   0.0,   0.0],
[0.0, -1.0, 0.0,  -0.08,   0.0,   0.0],
[0.0,  0.0, 1.0,    0.0,   0.0,   0.0]])

In [256]:
theta = np.array([[0],
                  [0],
                  [PI_HALF],
                  [0],
                  [0],
                  [0]])

alpha = np.array([[0],
                  [-PI_HALF],
                  [0],
                  [-PI_HALF],
                  [PI_HALF],
                  [-PI_HALF]])

d = np.array([[0],
              [kr6.d1],
              [0],
              [0],
              [kr6.d3],
              [kr6.d4]])

a = np.array([[0],
              [kr6.a1],
              [kr6.d1],
              [kr6.a2],
              [0],
              [0]])

dhList = np.concatenate((a, alpha, d, theta), axis=1)
dhList


array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.025     , -1.57079633,  0.4       ,  0.        ],
       [ 0.4       ,  0.        ,  0.        ,  1.57079633],
       [ 0.035     , -1.57079633,  0.        ,  0.        ],
       [ 0.        ,  1.57079633,  0.42      ,  0.        ],
       [ 0.        , -1.57079633,  0.08      ,  0.        ]])

In [257]:
l1 = DHLink(*dhList[0])
l2 = DHLink(*dhList[1])
l3 = DHLink(*dhList[2])
l4 = DHLink(*dhList[3])
l5 = DHLink(*dhList[4])
l6 = DHLink(*dhList[5])

In [258]:
fk_dh_zero = (
    l1.matrix()
    @ l2.matrix()
    @ l3.matrix()
    @ l4.matrix()
    @ l5.matrix()
    @ l6.matrix()
)
fk_dh_zero


array([[ 0.   , -0.   , -1.   , -0.395],
       [ 0.   , -1.   ,  0.   ,  0.08 ],
       [-1.   , -0.   , -0.   , -0.035],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])

In [259]:
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes, Ball
from pythreejs import Object3D
from transformations import quaternion_from_matrix
from pythreejs import ConeGeometry
from pythreejs import Preview

In [260]:
class DHFrame(Axes):
    def __init__(self, trf):
        Axes.__init__(self, 0.1)
        self.quaternion = np.roll(quaternion_from_matrix(trf), -1).tolist()
        self.position = (trf[:3, 3]).tolist()

In [261]:
viewer = Viewer(background="white")
viewer.add(Grid())

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [262]:
T01 = l1.matrix()
T02 = l1.matrix() @ l2.matrix()
T03 = l1.matrix() @ l2.matrix() @ l3.matrix()
T04 = l1.matrix() @ l2.matrix() @ l3.matrix() @ l4.matrix()
T05 = l1.matrix() @ l2.matrix() @ l3.matrix() @ l4.matrix() @ l5.matrix()
T06 = l1.matrix() @ l2.matrix() @ l3.matrix() @ l4.matrix() @ l5.matrix() @ l6.matrix()

T01

array([[ 1., -0.,  0.,  0.],
       [ 0.,  1., -0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [263]:
viewer.add(Axes(0.5))

In [264]:
viewer.add(DHFrame(T01))

In [265]:
viewer.add(DHFrame(T02))

In [266]:
viewer.add(DHFrame(T03))

In [267]:
viewer.add(DHFrame(T04))

In [268]:
viewer.add(DHFrame(T05))

In [269]:
viewer.add(DHFrame(T06))